In [86]:
# libraries
import plotly
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

warnings.simplefilter(action='ignore', category=FutureWarning) # ignores warnings from pd.append()

In [3]:
# import csv
df = pd.read_csv('dff.csv')
df.tail()

,Titel,Bevilliget beløb,Modtager,Institution,Virkemidler,Område,År,Beskrivelse,Region
4385,Enabling Ultra Deep Hydrodesulphurization by N...,10781874,Ib Chorkendorff,Danmarks Tekniske Universitet,Øvrige forskningsprojekter,Teknologi og Produktion,2013,Alle olieprodukter renses i dag for svovl for ...,Region Hovedstaden
4386,Acute stroke research,717359,Hanne Krarup Christensen,"Bispebjerg Hospital, Neurologisk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Aktuelle ansøgning angår frikøb af overlæge Ha...,Region Hovedstaden
4387,Atherosclerotic cardiovascular disease in HIV-...,764683,Anne-Mette Lebech,"Hvidovre Hospital, Infektionsmedicinsk Afdeling",Delestillinger,Sundhed og Sygdom,2013,Behandling af HIV positive patienter med anti-...,Region Hovedstaden
4388,Epigenetic modulation of mechanisms involved i...,829294,Ole Schmeltz Søgaard,Aarhus Universitetshospital,Delestillinger,Sundhed og Sygdom,2013,HIV infektion behandles i dag med en kombinati...,Region Midtjylland
4389,Novel mechanisms of insulin resistance and mit...,665923,Kurt Højlund,"Odense Universitetshospital, Endokrinologisk A...",Delestillinger,Sundhed og Sygdom,2013,Insulinresistens (IR) i muskelvæv spiller en v...,Region Syddanmark


In [75]:
# colors for fun
print(px.colors.sequential.Plotly3)

#fig = px.colors.sequential.swatches_continuous()
#fig.show()

['#0508b8', '#1910d8', '#3c19f0', '#6b1cfb', '#981cfd', '#bf1cfd', '#dd2bfd', '#f246fe', '#fc67fd', '#fe88fc', '#fea5fd', '#febefe', '#fec3fe']


In [ ]:
df_sankey = df.groupby(['År','Virkemidler','Region', 'Område']).agg({'Bevilliget beløb':'sum'}).reset_index()

In [99]:
def generateSankey(df, year, cat_cols=[]):
    
    # colors
    colorPalette = px.colors.sequential.Plotly3

    # data for sankey
    df_sankey = df.groupby(cat_cols).agg({'Bevilliget beløb':'sum'}).reset_index()
    df_sankey = df_sankey[df_sankey.År == year]

    # create list of labels, i.e. unique values from each column except the values
    labels = []

    for col in df_sankey.loc[:, df_sankey.columns != 'Bevilliget beløb']:
        labels = labels + (list(set(df_sankey[col].values)))

    # initiate input for for loop
    category_columns = cat_cols # len should be at least to, otherwise data cannot flow from category 1 to category 2.
    value_column = 'Bevilliget beløb'
    df_link_input = pd.DataFrame({'source' : [], 'target': [], 'count': []})

    # create data for go.Sankey function
    for i in range(len(category_columns)-1):
        if len(category_columns) == 1:
            print("Number of input categories must be at least 2")
        else:
            temporary_df = df_sankey.groupby([category_columns[i], category_columns[i+1]]).agg({value_column:'sum'}).reset_index() # loop over columns and group by column to the right, i.e. 'År' and 'Virkemidler', and then 'Virkemidler' and 'Område'
            temporary_df.columns = ['source','target','count']
            df_link_input = df_link_input.append(temporary_df)

    # add index for source-target pair
    df_link_input['sourceID'] = df_link_input['source'].apply(lambda x: labels.index(x))
    df_link_input['targetID'] = df_link_input['target'].apply(lambda x: labels.index(x))

    # creating the sankey diagram
    fig = go.Figure(data=[go.Sankey(
        valuesuffix = " DKK",
        # define nodes
        node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = labels,
        color = "blue"
        ),
        link = dict(
        source = df_link_input['sourceID'], # indices correspond to labels, e.g. '2022', 'Forskningsprojekt 1', 'Forskningsprojekt 2', ...
        target = df_link_input['targetID'],
        value = df_link_input['count']
    ))])

    fig.update_layout(title_text="Funding of Research Grants in " + str(year) + "<br>Source: <a href='https://dff.dk/'>Danmarks Frie Forskningsfond</a>",
                    font_size=10)
    fig.show()

    print(df_link_input["count"].sum())

In [103]:
# sankey diagram example
generateSankey(df, 2021, cat_cols = ['År', 'Virkemidler', 'Område'])

# !!!!!!!!!!!!!!! SANKEY DIAGRAM FRA NETTET OG EGEN FUNKTION SUMMER IKKE TIL DET SAMME FOR ÅRET 2021!!!!!!!!!!!!!!!
# NODES UDE TIL HØJRE SKAL SORTERES I FALDENDE ORDEN
# 

2882966316.0


In [101]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp   

    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))

    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
    
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    print(sourceTargetDf)

    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
    print(sourceTargetDf["count"].sum())
    fig = dict(data=[data], layout=layout)
    return fig

In [104]:
# data for sankey plot
gk = df.groupby(['År','Virkemidler','Region', 'Område']).agg({'Bevilliget beløb':'sum'})
gk = gk.reset_index()
gk
df_sankey = gk[gk.År == 2021]

fig = genSankey(df,cat_cols=['År', 'Virkemidler', 'Område'], value_cols='Bevilliget beløb', title='Titel')
plotly.offline.plot(fig, validate=False)

                         source                                      target  \
0                          2013                              Delestillinger   
1                          2013                              Forskernetværk   
2                          2013  Forskeruddannelse uden for universiteterne   
3                          2013                 Forskningsophold i udlandet   
4                          2013                         Forskningsprojekt 1   
..                          ...                                         ...   
242  Øvrige forskningsprojekter                          Samfund og Erhverv   
243  Øvrige forskningsprojekter                           Sundhed og Sygdom   
244  Øvrige forskningsprojekter                     Teknologi og Produktion   
245            Øvrige projekter                     Teknologi og Produktion   
246            Øvrige projekter                                 Tværrådslig   

         count  sourceID  targetID  
0      2977259

'temp-plot.html'